# Initialize

In [8]:
from src.data.dump_data import load_orders_data
load_orders_data()

,symbol,orderId,orderListId,clientOrderId,price,origQty,executedQty,cummulativeQuoteQty,status,timeInForce,type,side,stopPrice,icebergQty,time,updateTime,isWorking,origQuoteOrderQty,base_coin,quote_coin,date
0,SOLUSDT,3.250025e+08,-1.0,ios_ab3cb1da3d7d4d12aa9c44528512c70b,41.00000,0.268000,0.268000,10.988000,FILLED,GTC,LIMIT,BUY,0.0,0.0,1621031032224,1621061234382,1.0,0.0,SOL,USDT,2021-05-15 01:23:52.224
1,BNBUSDT,2.203228e+09,-1.0,ios_f92ba5be0cd142b0b4053b3662201076,500.00000,0.020000,0.020000,10.000000,FILLED,GTC,LIMIT,BUY,0.0,0.0,1621333247911,1621348494879,1.0,0.0,BNB,USDT,2021-05-18 13:20:47.911
2,BTCUSDT,6.002429e+09,-1.0,ios_1c3036291afd4846bbdf7b281a50af9b,39200.00000,0.000316,0.000316,12.387200,FILLED,GTC,LIMIT,BUY,0.0,0.0,1621406299240,1621406765597,1.0,0.0,BTC,USDT,2021-05-19 09:38:19.240
3,ETHUSDT,4.132587e+09,-1.0,ios_e70d9b3d520f4118aee596d465d9c73b,2900.00000,0.003450,0.003450,10.005000,FILLED,GTC,LIMIT,BUY,0.0,0.0,1621406408728,1621408328111,1.0,0.0,ETH,USDT,2021-05-19 09:40:08.728
4,SOLBUSD,1.076348e+08,-1.0,ios_2bbf68e8c2a4432ab9c2de475a9a9f02,45.00000,0.273000,0.273000,12.285000,FILLED,GTC,LIMIT,BUY,0.0,0.0,1621408334746,1621423102127,1.0,0.0,SOL,BUSD,2021-05-19 10:12:14.746
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,ETHUSDT,7.542362e+09,-1.0,and_b8f2b7a7ada346eb9a473f5d720ea9ad,2455.55000,0.021400,0.021400,52.536144,FILLED,GTC,LIMIT,BUY,0.0,0.0,1643005599630,1643005599630,1.0,0.0,ETH,USDT,2022-01-24 09:26:39.630
95,ETHUSDT,7.542414e+09,-1.0,and_9a8b1f9956744ef6a28a6cbef96ac088,2446.17000,0.028600,0.028600,69.960462,FILLED,GTC,LIMIT,BUY,0.0,0.0,1643005953512,1643005981095,1.0,0.0,ETH,USDT,2022-01-24 09:32:33.512
96,XRPUSDT,3.922866e+09,-1.0,and_45f59b2a129b425f818656ec976c06b4,0.60850,94.000000,94.000000,57.199000,FILLED,GTC,LIMIT,BUY,0.0,0.0,1643005987750,1643005990683,1.0,0.0,XRP,USDT,2022-01-24 09:33:07.750
97,CELRUSDT,4.246271e+08,-1.0,and_a18eb068e2f444d1ac46f80e2ebad972,0.04893,501.100000,501.100000,24.473724,FILLED,GTC,LIMIT,BUY,0.0,0.0,1643048782264,1643048782264,1.0,0.0,CELR,USDT,2022-01-24 21:26:22.264


In [3]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../')
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
pd.options.display.max_columns=None
from src.client.client import ClientHelper
from src.analysis.analyse import OrdersAnalyser, AssetAnalyser
from src.utils.utils import load_api_keys_from_json

In [ ]:
api_key, api_secret = load_api_keys_from_json('../api_key.json')
client_helper = ClientHelper(api_key, api_secret)

# Analyze portfolio

In [ ]:
prices = client_helper.query_prices()
history_assets = client_helper.get_history_assets('SPOT', days=30)
asset_analyser = AssetAnalyser(client_helper)
portfolio_fig = asset_analyser.plot_asset_composition_in_usdt(history_assets, prices)
portfolio_fig

In [ ]:
asset_history_fig = asset_analyser.plot_asset_usdt_value_history(history_assets)
asset_history_fig

# Analyze orders history

In [ ]:
%%time
orders = client_helper.get_all_orders()
orders.head()

In [ ]:
order_analyser = OrdersAnalyser(client_helper, orders)
mean_price = order_analyser.calculate_mean_price()
mean_price = pd.merge(mean_price, prices.rename(columns={'price':'current_price'}), on=['base_coin','quote_coin'], how='left')
mean_price['price_change_usd'] = mean_price['current_price'] - mean_price['average_price']
mean_price['price_change_percent'] = mean_price['price_change_usd']/mean_price['average_price']*100
display(mean_price.round(3))

In [ ]:
for base_coin in ['CELR', 'ETH']:
    price_history = client_helper.get_historical_prices(base_coin+'USDT', start_date='1 Jan, 2021')
    fig = order_analyser.plot_transactions(base_coin, price_history)
    fig.show()

# Trade

In [ ]:

# get market depth
depth = client.get_order_book(symbol='BNBBTC')

# place a test market buy order, to place an actual order use the create_order function
order = client.create_test_order(
    symbol='BNBBTC',
    side=Client.SIDE_BUY,
    type=Client.ORDER_TYPE_MARKET,
    quantity=100)

# get all symbol prices
prices = client.get_all_tickers()

# withdraw 100 ETH
# check docs for assumptions around withdrawals
from binance.exceptions import BinanceAPIException
try:
    result = client.withdraw(
        asset='ETH',
        address='<eth_address>',
        amount=100)
except BinanceAPIException as e:
    print(e)
else:
    print("Success")

# fetch list of withdrawals
withdraws = client.get_withdraw_history()

# fetch list of ETH withdrawals
eth_withdraws = client.get_withdraw_history(coin='ETH')

# get a deposit address for BTC
address = client.get_deposit_address(coin='BTC')

# get historical kline data from any date range

# fetch 1 minute klines for the last day up until now
klines = client.get_historical_klines("BNBBTC", Client.KLINE_INTERVAL_1MINUTE, "1 day ago UTC")

# fetch 30 minute klines for the last month of 2017
klines = client.get_historical_klines("ETHBTC", Client.KLINE_INTERVAL_30MINUTE, "1 Dec, 2017", "1 Jan, 2018")

# fetch weekly klines since it listed
klines = client.get_historical_klines("NEOBTC", Client.KLINE_INTERVAL_1WEEK, "1 Jan, 2017")

# socket manager using threads
twm = ThreadedWebsocketManager()
twm.start()

# depth cache manager using threads
dcm = ThreadedDepthCacheManager()
dcm.start()

def handle_socket_message(msg):
    print(f"message type: {msg['e']}")
    print(msg)

def handle_dcm_message(depth_cache):
    print(f"symbol {depth_cache.symbol}")
    print("top 5 bids")
    print(depth_cache.get_bids()[:5])
    print("top 5 asks")
    print(depth_cache.get_asks()[:5])
    print("last update time {}".format(depth_cache.update_time))

twm.start_kline_socket(callback=handle_socket_message, symbol='BNBBTC')

dcm.start_depth_cache(callback=handle_dcm_message, symbol='ETHBTC')

# replace with a current options symbol
options_symbol = 'BTC-210430-36000-C'
dcm.start_options_depth_cache(callback=handle_dcm_message, symbol=options_symbol)

# join the threaded managers to the main thread
twm.join()
dcm.join()